In [3]:
import vtk
from vtk.util import numpy_support
from vtk.numpy_interface import dataset_adapter
from pbcpy.formats.qepp import PP
import numpy as np

In [95]:
dataset_adapter

<module 'vtk.numpy_interface.dataset_adapter' from '/home/alessandro/anaconda3/lib/python3.6/site-packages/vtk/numpy_interface/dataset_adapter.py'>

In [4]:
rho = PP("/home/alessandro/QE/FDE_Calc/dimer/water-trimer_fde_rho_2.pp").read()

In [30]:
field = rho.field.ravel()
rho.field.shape

(120, 120, 120, 1)

In [31]:
field_vtk = numpy_support.numpy_to_vtk(field, deep=True)
field_vtk

(vtkCommonCorePython.vtkDoubleArray)0x7f0ab3015e88

In [85]:
iD = vtk.vtkImageData()

iD.SetOrigin(0.,0.,0.)
iD.SetSpacing(1.,1.,1.)
iD.SetDimensions((120,60,240))

#iD.GetPointData().SetScalars(field_vtk)

geom = vtk.vtkGeometryFilter()
#geom = vtk.vtk()
geom.SetInputData(iD)

#m = vtk.vtkPolyDataMapper()
m = vtk.vtkDataSetMapper()
m.SetInputConnection(geom.GetOutputPort())

a = vtk.vtkActor()
a.SetMapper(m)
#geom

In [86]:
ren, win, iren = init_window()

In [87]:
ren.AddActor(a)

In [88]:
iren.Initialize()
win.Render()
iren.Start()

In [89]:
win.Finalize()
iren.TerminateApp()
del win, iren

In [15]:
data_vtk = vtk.
data_vtk.SetDimensions((120,120,120))
#img_vtk.SetSpacing(spacing[::-1])
data_vtk.GetPointData().SetScalars(field_vtk)

AttributeError: 'vtkRenderingCorePython.vtkDataSetMapper' object has no attribute 'SetDimensions'

In [14]:
from vtk.util.colors import tomato

# This creates a polygonal cylinder model with eight circumferential
# facets.
cylinder = vtk.vtkCylinderSource()
cylinder.SetResolution(8)

# The mapper is responsible for pushing the geometry into the graphics
# library. It may also do color mapping, if scalars or other
# attributes are defined.
cylinderMapper = vtk.vtkPolyDataMapper()
cylinderMapper.SetInputConnection(cylinder.GetOutputPort())

# The actor is a grouping mechanism: besides the geometry (mapper), it
# also has a property, transformation matrix, and/or texture map.
# Here we set its color and rotate it -22.5 degrees.
cylinderActor = vtk.vtkActor()
cylinderActor.SetMapper(cylinderMapper)
#cylinderActor.SetMapper(data_vtk)
prop = cylinderActor.GetProperty()
prop.SetColor()
cylinderActor.RotateX(30.0)
cylinderActor.RotateY(-45.0)

In [13]:
# Create the graphics structure. The renderer renders into the render
# window. The render window interactor captures mouse events and will
# perform appropriate camera or actor manipulation depending on the
# nature of the events.
ren = vtk.vtkRenderer()
renWin = vtk.vtkRenderWindow()
renWin.AddRenderer(ren)
iren = vtk.vtkRenderWindowInteractor()
iren.SetRenderWindow(renWin)

In [14]:
# Add the actors to the renderer, set the background and size
ren.AddActor(cylinderActor)
ren.SetBackground(0.1, 0.2, 0.4)
renWin.SetSize(200, 200)

# This allows the interactor to initalize itself. It has to be
# called before an event loop.
iren.Initialize()

# We'll zoom in a little by accessing the camera and invoking a "Zoom"
# method on it.
ren.ResetCamera()
ren.GetActiveCamera().Zoom(1.5)
renWin.Render()

# Start the event loop.
iren.Start()

In [104]:
def init_window():
    import vtk
    # Create the graphics structure. The renderer renders into the render
    # window. The render window interactor captures mouse events and will
    # perform appropriate camera or actor manipulation depending on the
    # nature of the events.
    ren = vtk.vtkRenderer()
    renWin = vtk.vtkRenderWindow()
    renWin.AddRenderer(ren)
    iren = vtk.vtkRenderWindowInteractor()
    iren.SetRenderWindow(renWin)
    # Add the actors to the renderer, set the background and size
    #ren.AddActor(cylinderActor)
    ren.SetBackground(1, 1, 1)
    renWin.SetSize(400, 400)

    # This allows the interactor to initalize itself. It has to be
    # called before an event loop.
    #iren.Initialize()

    # We'll zoom in a little by accessing the camera and invoking a "Zoom"
    # method on it.
    ren.ResetCamera()
    #ren.GetActiveCamera().Zoom(1.5)
    # Start the event loop.
    #iren.Start()
    
    return ren, renWin, iren


def add_field(field, renderer, iso=0.1, k=0):
    import vtk
    from vtk.numpy_interface import dataset_adapter
    
    spacing = np.zeros(3)
    for i in range(3):
        spacing[i] = field.grid.lattice[i,i]/field.grid.nr[i]
    
    iD = vtk.vtkImageData()
    
    field_vtk = numpy_support.numpy_to_vtk(field.ravel(order="F"), deep=True)    
    iD.GetPointData().SetScalars(field_vtk)
    iD.SetOrigin(0.,0.,0.)
    iD.SetSpacing(spacing)
    iD.SetDimensions(field.grid.nr)
   
    contour = vtk.vtkMarchingCubes()
    contour.SetValue(0, iso)
    contour.SetInputData(iD)
   
    
    #stripper = vtk.vtkStripper()
    #stripper.SetInputConnection(contour.GetOutputPort())

    m = vtk.vtkPolyDataMapper()
    m.SetInputConnection(contour.GetOutputPort())
    m.Update()
    m.ScalarVisibilityOff()
    #m.SetInputData(iD)
    
    a = vtk.vtkActor()
    a.SetMapper(m)
    a.GetProperty().SetDiffuseColor(i2color(k))
    a.GetProperty().SetOpacity(0.4)
    #a.GetProperty().EdgeVisibilityOn()
    
    renderer.AddActor(a)


def add_atom(label, pos, renderer):
    r = 1.0
    ball = vtk.vtkSphereSource()
    ball.SetRadius(label2radius(label))
    ball.SetCenter(pos)

    # The mapper is responsible for pushing the geometry into the graphics
    # library. It may also do color mapping, if scalars or other
    # attributes are defined.
    ballMapper = vtk.vtkPolyDataMapper()
    ballMapper.SetInputConnection(ball.GetOutputPort())

    # The actor is a grouping mechanism: besides the geometry (mapper), it
    # also has a property, transformation matrix, and/or texture map.
    # Here we set its color and rotate it -22.5 degrees.
    ballActor = vtk.vtkActor()
    ballActor.SetMapper(ballMapper)
    #cylinderActor.SetMapper(data_vtk)
    ballActor.GetProperty().SetColor(label2color(label))
    
    renderer.AddActor(ballActor)

    
def label2color(label):
    color_dict = {
        "O": (1.,0.,0.),
        "H": (0.9,0.9,0.9)
    }
    if label in color_dict:
        return color_dict[label]
    return (.5,.5,.5)

def label2radius(label):
    radius_dict = {
        "O": 0.5,
        "H": 0.25
    }
    if label in radius_dict:
        return radius_dict[label]
    return 0.5

def i2color(i):
    n = 6
    j = i%n
    colors = [
        (1.,0.,0.),
        (0.,1.,0.),
        (0.,0.,1.),
        (1.,1.,0.),
        (1.,0.,1.),
        (0.,1.,1.),
    ]
    return colors[j]

def render_pp(filename):
    from pbcpy.formats.qepp import PP
    system = PP(filename).read()
    render_system(system)
    
def render_system(system):
    ren, win, iren = init_window()
    
    for atom in system.ions:
        add_atom(atom.label, atom.pos, ren)
        
    iso = 0.1
    i = 6
    add_field(system.field, ren, iso, i)
    
    iren.Initialize()
    win.Render()
    iren.Start()
    
    win.Finalize()
    iren.TerminateApp()
    del win, iren

In [105]:
render_pp("/home/alessandro/QE/FDE_Calc/dimer/water-trimer_fde_rho_2.pp")

In [93]:
iren, win, iren = init_window()

In [98]:
for atom in rho.ions:
    add_atom(atom.label, atom.pos, ren)
    #print(atom.label, atom.pos)
    


O [ 8.5037676   8.55442386  7.89977265]
H [ 7.05008861  8.58458885  9.02692964]
H [ 9.95744659  8.58458885  9.02692964]


In [94]:
iso = 0.01
i = 6
add_field(rho.field, ren, iso, i)

In [95]:
iren.Initialize()
win.Render()
iren.Start()

In [96]:
win.Finalize()
iren.TerminateApp()
del win, iren

In [13]:
actors = ren.Get

In [14]:
actors.

TypeError: 'vtkRenderingCorePython.vtkActorCollection' object is not subscriptable

In [6]:
ball = vtk.vtkSphereSource()
ball.SetRadius(1)
#ball.SetResolution(16)

# The mapper is responsible for pushing the geometry into the graphics
# library. It may also do color mapping, if scalars or other
# attributes are defined.
ballMapper = vtk.vtkPolyDataMapper()
ballMapper.SetInputConnection(ball.GetOutputPort())

# The actor is a grouping mechanism: besides the geometry (mapper), it
# also has a property, transformation matrix, and/or texture map.
# Here we set its color and rotate it -22.5 degrees.
ballActor = vtk.vtkActor()
ballActor.SetMapper(ballMapper)
ballActor.SetOrigin()
#cylinderActor.SetMapper(data_vtk)
#ballActor.GetProperty().SetColor(tomato)

#renderer.AddActor(ballActor)

In [30]:
rho.field.min(), rho.field.max()


(DirectField(-7.471154341e-08), DirectField(1.055862139))

In [23]:
def sin_field(g, nr):
    field = np.zeros((nr,nr,nr), dtype=np.uint8)
    for i in range(nr):
        field[i,:,:] = (1+np.sin(2*np.pi*i*g/nr))*127
    return field

In [24]:
from pbcpy.field import DirectField
from pbcpy.grid import DirectGrid
import numpy as np
nr = 90
grid = DirectGrid(np.identity(3),[nr,nr,nr])
field = DirectField(grid=grid, griddata_3d=sin_field(3, nr))

In [58]:
field[:,20,0]

DirectField([[127],
       [153],
       [178],
       [201],
       [221],
       [236],
       [247],
       [253],
       [253],
       [247],
       [236],
       [221],
       [201],
       [178],
       [153],
       [127],
       [100],
       [ 75],
       [ 52],
       [ 32],
       [ 17],
       [  6],
       [  0],
       [  0],
       [  6],
       [ 17],
       [ 32],
       [ 52],
       [ 75],
       [100],
       [126],
       [153],
       [178],
       [201],
       [221],
       [236],
       [247],
       [253],
       [253],
       [247],
       [236],
       [221],
       [201],
       [178],
       [153],
       [127],
       [100],
       [ 75],
       [ 52],
       [ 32],
       [ 17],
       [  6],
       [  0],
       [  0],
       [  6],
       [ 17],
       [ 32],
       [ 52],
       [ 75],
       [100],
       [126],
       [153],
       [178],
       [201],
       [221],
       [236],
       [247],
       [253],
       [253],
       [247],
       [236],


In [7]:
s = "abcd"

In [5]:
def reverse(s, i=0):
    if i >= len(s): return ""
    return reverse(s, i+1) + s[i]

In [8]:
reverse(s)

'dcba'

In [1]:
import numpy as np

In [20]:
filename = "/home/alessandro/QE/FDE_Calc/trimer/kitware/trimer.xyz"

data = np.genfromtxt(filename, dtype=str)
labels = data[:,0]
positions = np.array(data[:,1:], dtype=float)

In [17]:
def centerPos(pos, cell):
    cell = np.ones(3)*12.
    size = np.zeros(3)
    shift = np.zeros(3)
    max_pos = np.zeros(3)
    min_pos = np.zeros(3)
    
    for i in range(3):
        min_pos[i] = np.min(pos[:,i])
        max_pos[i] = np.max(pos[:,i])
        size[i] = max_pos[i] - min_pos[i]
        shift[i] = (cell[i]-size[i])/2. - min_pos[i]
        
    return size, shift

In [19]:
size, shift = centerPos(positions,"")

In [22]:
for label, pos in zip(labels, positions):
    pos += shift
    print("{}  {}  {}  {}".format(label, pos[0], pos[1], pos[2]))

O  6.0  6.0  4.755614920000001
H  6.0  6.77162768  5.360155960000001
H  6.0  5.22837232  5.360155960000001
O  6.0  3.7018061400000004  6.647919280000001
H  5.2307462000000005  3.71776877  7.24438508
H  6.7692537999999995  3.71776877  7.24438508
O  6.0  8.29819386  6.647919280000001
H  5.2307462000000005  8.28223123  7.24438508
H  6.7692537999999995  8.28223123  7.24438508


In [31]:
from collections import OrderedDict
a = OrderedDict()
a["asd"] = 1
a["asad"] = 1

In [33]:
folder = "/home/alessandro/code/pbcpy-qt/trimer/fde"

In [34]:
import os

In [42]:
for f in os.listdir(folder):
    if f.endswith(".pp"):
        print(os.path.join(folder, f))

/home/alessandro/code/pbcpy-qt/trimer/fde/density_1.pp
/home/alessandro/code/pbcpy-qt/trimer/fde/density_2.pp
/home/alessandro/code/pbcpy-qt/trimer/fde/density_0.pp


In [38]:
s = "asdsa.pp"


In [47]:
os.path.isdir(folder), os.path.isfile(folder+"/density_0.pp"), 

(True, True)